# Making CMIP6 multimodel means

### Get the CMIP6 catalogue

In [1]:
from intake import open_catalog

cat = open_catalog(
    "https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/master.yaml"
)["climate"]["cmip6_gcs"]
#list(cat)

### Pick out the historical temperature of the surface

In [13]:
import matplotlib.pyplot as plt
import xarray as xr
import dask
from cmip6_preprocessing.preprocessing import (
    rename_cmip6, 
    promote_empty_dims, 
    broadcast_lonlat, 
    replace_x_y_nominal_lat_lon,
    combined_preprocessing,
)

def get_ts(experiment="historical", year_begin="1958", year_end="2014"):
    query = dict(
        variable_id=["ts"],
        experiment_id=[experiment],# , "ssp585"],
        table_id=["Amon"],
        institution_id=["NCAR", "NOAA-GFDL", "MOHC", "NASA-GISS", "UA", "INM", "THU", "SNU", "E3SM-Project"],
    )
    subset = cat.search(**query)

    z_kwargs = {"consolidated": True, "decode_times": True}

    def wrapper(ds):
        ds = ds.copy()
        ds = rename_cmip6(ds)
        ds = promote_empty_dims(ds)
        ds = broadcast_lonlat(ds)
        ds = replace_x_y_nominal_lat_lon(ds)
        return ds

    # pass the preprocessing directly
    with dask.config.set(**{"array.slicing.split_large_chunks": True}):
        dset_dict_proc = subset.to_dataset_dict(zarr_kwargs=z_kwargs,
                                            preprocess=wrapper)
        
    da_list = []; key_list = []
    
    first = True

    for key in dset_dict_proc:
        print(key)
        da = dset_dict_proc[key].ts.mean("member_id").sel(
            x=slice(100, 300), y=slice(-30, 30), time=slice(year_begin, year_end)
        ).interp(x=list(range(100, 300)), y=list(range(-30, 30)))  # .mean("time").plot()
        key_list.append(key)
        if first:
            times = da.time.values
        da = da.assign_coords(time=list(range(da.sizes["time"])))
        da_list.append(da)
    da = xr.concat(da_list, "model_center")
    da = da.assign_coords({"model_center": key_list, "time": times})
    
    return da

In [17]:
da_hist = get_ts()
da_ssp585 = get_ts(experiment="ssp585", year_begin="2014", year_end="2017")
combined_da_60 = xr.concat([da_hist.mean("model_center"), da_ssp585.mean("model_center")], "time")


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


CMIP.NCAR.CESM2-WACCM-FV2.historical.Amon.gn
CMIP.MOHC.HadGEM3-GC31-MM.historical.Amon.gn
CMIP.SNU.SAM0-UNICON.historical.Amon.gn
CMIP.NOAA-GFDL.GFDL-CM4.historical.Amon.gr1
CMIP.THU.CIESM.historical.Amon.gr
CMIP.MOHC.UKESM1-0-LL.historical.Amon.gn
CMIP.NASA-GISS.GISS-E2-1-H.historical.Amon.gn
CMIP.NASA-GISS.GISS-E2-1-G-CC.historical.Amon.gn
CMIP.E3SM-Project.E3SM-1-1.historical.Amon.gr
CMIP.E3SM-Project.E3SM-1-0.historical.Amon.gr
CMIP.NCAR.CESM2-FV2.historical.Amon.gn
CMIP.E3SM-Project.E3SM-1-1-ECA.historical.Amon.gr
CMIP.NOAA-GFDL.GFDL-ESM4.historical.Amon.gr1
CMIP.INM.INM-CM5-0.historical.Amon.gr1
CMIP.INM.INM-CM4-8.historical.Amon.gr1
CMIP.MOHC.HadGEM3-GC31-LL.historical.Amon.gn
CMIP.UA.MCM-UA-1-0.historical.Amon.gn
CMIP.NCAR.CESM2.historical.Amon.gn
CMIP.NCAR.CESM2-WACCM.historical.Amon.gn
CMIP.NASA-GISS.GISS-E2-1-G.historical.Amon.gn

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid

ScenarioMIP.UA.MCM-UA-1-0.ssp585.Amon.gn
ScenarioMIP.MOHC.HadGEM3-GC31-LL.ssp585.Amon.gn
ScenarioMIP.NOAA-GFDL.GFDL-CM4.ssp585.Amon.gr1
ScenarioMIP.MOHC.UKESM1-0-LL.ssp585.Amon.gn
ScenarioMIP.THU.CIESM.ssp585.Amon.gr
ScenarioMIP.INM.INM-CM4-8.ssp585.Amon.gr1
ScenarioMIP.E3SM-Project.E3SM-1-1.ssp585.Amon.gr
ScenarioMIP.MOHC.HadGEM3-GC31-MM.ssp585.Amon.gn
ScenarioMIP.NCAR.CESM2.ssp585.Amon.gn
ScenarioMIP.NOAA-GFDL.GFDL-ESM4.ssp585.Amon.gr1
ScenarioMIP.NCAR.CESM2-WACCM.ssp585.Amon.gn
ScenarioMIP.INM.INM-CM5-0.ssp585.Amon.gr1
ScenarioMIP.NASA-GISS.GISS-E2-1-G.ssp585.Amon.gn


In [ ]:
combined_da_60.isel(time=0).plot()

60.0

In [ ]:
da.mean("model_center").mean("time").plot()

In [29]:
da_list[0].time.values

array([cftime.DatetimeNoLeap(1954, 1, 15, 12, 0, 0, 0),
       cftime.DatetimeNoLeap(1954, 2, 14, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(1954, 3, 15, 12, 0, 0, 0),
       cftime.DatetimeNoLeap(1954, 4, 15, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(1954, 5, 15, 12, 0, 0, 0),
       cftime.DatetimeNoLeap(1954, 6, 15, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(1954, 7, 15, 12, 0, 0, 0),
       cftime.DatetimeNoLeap(1954, 8, 15, 12, 0, 0, 0),
       cftime.DatetimeNoLeap(1954, 9, 15, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(1954, 10, 15, 12, 0, 0, 0),
       cftime.DatetimeNoLeap(1954, 11, 15, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(1954, 12, 15, 12, 0, 0, 0),
       cftime.DatetimeNoLeap(1955, 1, 15, 12, 0, 0, 0),
       cftime.DatetimeNoLeap(1955, 2, 14, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(1955, 3, 15, 12, 0, 0, 0),
       cftime.DatetimeNoLeap(1955, 4, 15, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(1955, 5, 15, 12, 0, 0, 0),
       cftime.DatetimeNoLeap(1955, 6, 15, 0, 0, 0, 0